# ORS Assessment Test Automatic Grader
Automate grading assessment tests and output a report in excel format

**Requirments**
1. Get the correct answers of the assessment
1. Filter by student (also filter by datetime for current cohort)
1. Find the pre-class assessment
1. Find the post-class assessment
1. Find reattempts of tests
1. Show only answers of the questions students get wrong
1. Transpose from wide data to long data
  1. Column: Attempts (labels: pre-class test, post-class test, reattempts)
  1. Row: Scores, Questions 1 - 20
1. Save into `.xlsx` file

**Tasks**
- Provide dataset of the students (`.csv` or google sheets api)
- Provide student names

## Install Required Libraries & Import

In [252]:
import pandas as pd
import numpy as np
import os

## Create Folders & Upload Files
- Assessment test responses `.csv` file from this [link](https://drive.google.com/drive/folders/142C-KrYeCN2GnEUiyou6cvDYIOyOWKkt?usp=drive_link)

- Student information from this [link](https://nyplorg-my.sharepoint.com/:x:/g/personal/kangchen_nypl_org/EVcsHhpkqM9Fteg3pnvZBZoBRawkm43iJZF-3YE5hgRyCA?email=kangchen%40nypl.org&e=l5Vz5w)

In [253]:
cwd = os.getcwd()
data_dir = 'data'
if not os.path.exists(data_dir):
    path = os.path.join(cwd, data_dir)
    os.mkdir(os.path.join(cwd, data_dir))
    print('Folder created at: {}'.format(path))

## Set Variables

In [254]:
# set assessment dates
test_dates =['2023-09-13', '2023-09-16', '2023-10-14', '2023-10-28', '2023-11-04','2023-11-18', '2023-11-25', '2023-12-02']
test_dates = [pd.to_datetime(date) for date in test_dates]
print(test_dates)

[Timestamp('2023-09-13 00:00:00'), Timestamp('2023-09-16 00:00:00'), Timestamp('2023-10-14 00:00:00'), Timestamp('2023-10-28 00:00:00'), Timestamp('2023-11-04 00:00:00'), Timestamp('2023-11-18 00:00:00'), Timestamp('2023-11-25 00:00:00'), Timestamp('2023-12-02 00:00:00')]


In [255]:
# set what software the assessment is for
program = 'powerpoint'
student_csv = os.path.join(path, 'ors_stu_info.csv')
test_responses = os.path.join(path, f'ors_{program}_assessment_responses.csv')
print(test_responses)

/Users/kangchen/Documents/Projects/ors_auto_grading/data/ors_powerpoint_assessment_responses.csv


## Process Names

In [256]:
# set variables
df = pd.read_csv(student_csv)
df

,FirstName,LastName,Drop,Email,OneDrive,MosCertInterest,WordTest,ExcelTest,PPTTest
0,Alana,Chuong,NaN,alanachuong@gmail.com,alanachuong.works@gmail.com,True,75.0,84.0,NaN
1,Alison,Wong,NaN,awtextmessages@gmail.com,awtextmessages@gmail.com,True,95.0,88.0,NaN
2,Alona Brequillo,Alona Brequillo,NaN,brequillo.alona@gmail.com,brequillo.alona@gmail.com,True,85.0,80.0,NaN
3,Beverly,Sealey,NaN,bevlun7@gmail.com,bevlun36@hotmail.com,True,80.0,92.0,NaN
4,Cherie,Pepper,NaN,pepper_cherie@yahoo.com,cherie.pepper@yahoo.com,True,80.0,NaN,NaN
5,Eleanor,Sasso,NaN,candidateinformation@me.com,infoc7012@gmail.com,True,80.0,84.0,NaN
6,Elsa,Wilson,NaN,elsa.wilson220@gmail.com,elsa.wilson220@gmail.com,True,65.0,48.0,NaN
7,FELICIA,POOLER,NaN,financialpeace312@gmail.com,poolerfelicia1@outlook.com,True,85.0,84.0,NaN
8,Fran,Brown,True,franaltabrown@gmail.com,franaltamirbrown@outlook.com,True,80.0,NaN,NaN
9,John,Niland,NaN,johnnyspotone@gmail.com,JohnNiland100@outlook.com,True,100.0,96.0,NaN


### Preprocess names

In [257]:
# drop unactive students
df = df.loc[df.Drop != True]

In [258]:
student_info = df[['FirstName', 'LastName', 'Email']]

In [259]:
names = df[['FirstName', 'LastName']]

In [260]:
names.loc[names.FirstName == 'Eleanor', ['FirstName']] = 'E'

/var/folders/ms/xhs4ws9j0px0b4g7j1tw58_c0000gp/T/ipykernel_45217/3352764092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names.loc[names.FirstName == 'Eleanor', ['FirstName']] = 'E'


In [261]:
# fix error values firstname = lastname
names.iloc[2,0] =names.iloc[2][0].split(' ')[0]
names.iloc[2,1] =names.iloc[2][1].split(' ')[1]

/var/folders/ms/xhs4ws9j0px0b4g7j1tw58_c0000gp/T/ipykernel_45217/3524485991.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  names.iloc[2,0] =names.iloc[2][0].split(' ')[0]
/var/folders/ms/xhs4ws9j0px0b4g7j1tw58_c0000gp/T/ipykernel_45217/3524485991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names.iloc[2,0] =names.iloc[2][0].split(' ')[0]
/var/folders/ms/xhs4ws9j0px0b4g7j1tw58_c0000gp/T/ipykernel_45217/3524485991.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access 

In [262]:
names.FirstName = names.FirstName.str.lower().str.strip()
names.LastName = names.LastName.str.lower().str.replace(' ', '')

/var/folders/ms/xhs4ws9j0px0b4g7j1tw58_c0000gp/T/ipykernel_45217/4173222479.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names.FirstName = names.FirstName.str.lower().str.strip()
/var/folders/ms/xhs4ws9j0px0b4g7j1tw58_c0000gp/T/ipykernel_45217/4173222479.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names.LastName = names.LastName.str.lower().str.replace(' ', '')


In [263]:
fullnames = pd.DataFrame(names.FirstName + ' ' + names.LastName, columns=['fullname'])

In [264]:
fullnames.sort_values(by='fullname', inplace=True)

## Process Responses

In [265]:
test_df = pd.read_csv(test_responses)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 25 columns):
 #   Column                                                                                                                                                               Non-Null Count  Dtype 
---  ------                                                                                                                                                               --------------  ----- 
 0   Timestamp                                                                                                                                                            748 non-null    object
 1   Email Address                                                                                                                                                        748 non-null    object
 2   Score                                                                                                                 

In [266]:
# rename columns
col_names_to_replace = list(test_df.columns)
col_names = ['timestamp', 'email', 'score', 'firstname', 'lastname']

for i in range(1,len(col_names_to_replace)-4):
  col_names.append(f'Q{i}')

col_mapper = dict.fromkeys(col_names_to_replace)
for i, col in enumerate(col_names_to_replace):
  col_mapper[col] = col_names[i]

test_df.rename(columns=col_mapper, inplace=True)

### Preprocess first and last name

In [267]:
test_df.firstname = test_df.firstname.str.strip().str.lower()
test_df.lastname = test_df.lastname.str.strip().str.lower()
test_df.head(5)

,timestamp,email,score,firstname,lastname,Q1,Q2,Q3,Q4,Q5,...,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
0,9/14/2020 17:18:20,ooaakn@gmail.com,65 / 100,natanya,oakley,False,All of the above,Slide Master View,All of the above.,Text Box Tool,...,"Slide Sorter View, PowerPoint View, Reading Vi...",Section,True,Advanced Animation,True,Design Tab,None of the above.,It is used for reorganizing slides in a presen...,True,Ends the slide show.
1,9/14/2020 17:19:08,grosso.sharon28@gmail.com,60 / 100,sharon,grosso,False,Text Box,Slide Sorter view,All of the above.,Text Box Tool,...,"Normal View, Slide Sorter View, PowerPoint Vie...",Alignment,True,"Animation, Advanced Animation",True,View Tab,They are animation-like effects that occur in ...,It is used for viewing all sides in a presenta...,True,Ends the slide show.
2,9/14/2020 17:19:41,DIMARIMD@AOL.COM,65 / 100,connie,dimari,False,Text Box,Normal view,All of the above.,Text Box Tool,...,"Normal View, Slide Sorter View, PowerPoint Vie...",None of the above.,True,"Advanced Animation, Preview",True,Design Tab,They are animation-like effects that occur in ...,It is used for reorganizing slides in a presen...,True,Ends the slide show.
3,9/14/2020 17:21:12,karnoldnyc@gmail.com,55 / 100,katie,arnold,False,Text Box,Slide Master View,"Change the font type., Change the font size., ...",Line Tool,...,"Normal View, Slide Sorter View, PowerPoint Vie...",All of the above.,True,"Animation, Advanced Animation",True,Design Tab,They are animation-like effects that occur in ...,It is used for reorganizing slides in a presen...,True,Ends the slide show.
4,9/14/2020 17:21:51,fettmartin@gmail.com,40 / 100,martin,fett,True,Text Box,Slide Master View,All of the above.,Text Box Tool,...,"Normal View, PowerPoint View, Reading View, Sl...",Alignment,False,"Timing, Animation",False,View Tab,None of the above.,It is used for reorganizing slides in a presen...,True,Displays the first slide.


In [268]:
# correct error values: elsa divinagracia = elsa
test_df.loc[(test_df.lastname == 'wilson') & (test_df.firstname == 'elsa divinagracia'), 'firstname'] = 'elsa'

### Get Answers

In [269]:
answer_row = test_df[test_df.score == '100 / 100'].tail(1).reset_index(drop=True)
answer_row.iloc[:,:5] = np.nan
answer_key = answer_row.iloc[:,5:]
answer_dict = answer_key.to_dict(orient='records')[0]

In [270]:
answer_dict

{'Q1': False,
 'Q2': 'Text Box',
 'Q3': 'Slide Master view',
 'Q4': 'All of the above.',
 'Q5': 'All of the above',
 'Q6': True,
 'Q7': 'Click Add Animation on the Animations tab of the Ribbon.',
 'Q8': 'SmartArt Graphics',
 'Q9': 'Engage',
 'Q10': True,
 'Q11': 'Normal View, Slide Sorter View, Reading View, Slide Show View',
 'Q12': 'Section',
 'Q13': True,
 'Q14': 'Timing, Advanced Animation',
 'Q15': True,
 'Q16': 'Design Tab',
 'Q17': 'They are animation-like effects that occur in the Slide Show View when the user moves from one slide to the next.',
 'Q18': 'It is used for easily reorganizing slides in a presentation.',
 'Q19': True,
 'Q20': 'Ends the slide show.'}

### Get Data Base on Test Date and Student Name

In [271]:
# convert timestamp to datetime dtype
test_df.timestamp = pd.to_datetime(test_df.timestamp, format='%m/%d/%Y %H:%M:%S', errors='coerce')

In [272]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  748 non-null    datetime64[ns]
 1   email      748 non-null    object        
 2   score      748 non-null    object        
 3   firstname  748 non-null    object        
 4   lastname   748 non-null    object        
 5   Q1         748 non-null    bool          
 6   Q2         748 non-null    object        
 7   Q3         748 non-null    object        
 8   Q4         748 non-null    object        
 9   Q5         747 non-null    object        
 10  Q6         748 non-null    bool          
 11  Q7         748 non-null    object        
 12  Q8         748 non-null    object        
 13  Q9         748 non-null    object        
 14  Q10        748 non-null    bool          
 15  Q11        748 non-null    object        
 16  Q12        748 non-null    object        
 1

In [273]:
# get all tests from test dates
all_word_assessment = test_df[test_df.timestamp.dt.date.isin([d.date() for d in test_dates])]

In [274]:
# check student names and test count
all_word_assessment[all_word_assessment.firstname.isin(names.FirstName)].firstname.value_counts().sort_index()

firstname
alana        2
alison       2
alona        2
beverly      2
cherie       3
e            2
elsa         2
felicia      1
john         2
julia        2
mayra        3
myrna        2
orlando      2
robert       2
roselynne    2
sitara       3
Name: count, dtype: int64

In [275]:
# all section 4 word test data
sect4_word_test = all_word_assessment[all_word_assessment.firstname.isin(names.FirstName)]
sect4_word_test.reset_index(drop=True, inplace=True)

## Generate Excel Reports

### Filter Incorrect Answers

In [276]:
info_df = sect4_word_test.iloc[:,:5] # student info
responses_df = sect4_word_test.iloc[:,5:] # student response
wrong_answer_filter = responses_df != pd.concat([answer_key] * responses_df.shape[0], ignore_index=True)
wrong_answer_df = responses_df[wrong_answer_filter] # retain answer values that are incorrect

In [277]:
# final dataframe with all student information and the questions the got wrong
final_results = pd.concat([info_df, wrong_answer_df], axis=1)

In [278]:
final_results.head(2)

,timestamp,email,score,firstname,lastname,Q1,Q2,Q3,Q4,Q5,...,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
0,2023-09-13 12:24:38,jahm1123@gmail.com,50 / 100,julia,hernandez-mejia,True,All of the above,Slide Show view,NaN,NaN,...,"Windows View, Slide Sorter View, PowerPoint Vi...",All of the above.,NaN,"Timing, Preview",NaN,Insert Tab,NaN,It is used for presenting a PowerPoint present...,NaN,NaN
1,2023-09-16 12:26:29,romixyz@gmail.com,80 / 100,roselynne,mizner,NaN,NaN,NaN,NaN,NaN,...,"Normal View, Slide Sorter View, PowerPoint Vie...",NaN,NaN,"Timing, Preview",NaN,NaN,NaN,NaN,NaN,NaN


### Save All Results to Dictionary

In [279]:
cols_to_show_list = []
grades_dict = {}
cols_to_drop = ['timestamp', 'email', 'score', 'firstname', 'lastname']

# iterate through each row
for i, row in final_results.iterrows():
  cols = list(final_results.columns[row.notna()]) # columns that don't contain null values
  cols_to_show_list.append(cols)

  # student answers
  stu_name = row.firstname.strip() + " " + row.lastname.strip()
  test_time = str(row.timestamp)

  # check if name exists
  if grades_dict.get(stu_name) == None:
    grades_dict[stu_name] = {}
  if grades_dict[stu_name].get(test_time) == None:
    grades_dict[stu_name][test_time] = {
        'score': row.score.split(' / ')[0],
        'res': pd.DataFrame(row.loc[cols]).T.drop(columns=cols_to_drop),
        'ans': answer_row[cols].drop(columns=cols_to_drop)
    }
  else:
    print(row.timestamp)

### Get everyones score -> Save to one single excel file

In [280]:
pre_class_scores = []
post_class_scores = []

for name in fullnames.fullname.to_list():
  print(name)
  scores_list = []
  for i, tup in enumerate(grades_dict[name].items()):
    # get pre-class assessment score
    if i == 0:
      _, score_dict = tup
      pre_score = float(score_dict['score'])
      pre_class_scores.append(pre_score)
      print('pre-class:', pre_score)
      continue

    _, score_dict = tup
    scores_list.append(float(score_dict['score']))

  # get post-class assessment highest score
  if len(scores_list) == 0:
    post_class_scores.append(np.NaN)
    print('post-class:', np.NAN)
  else:
    highest_grade_idx = np.argmax(np.array(scores_list))
    highest_score = scores_list[highest_grade_idx]
    post_class_scores.append(highest_score)
    print('post-class:', highest_score)
  print('------')

alana chuong
pre-class: 55.0
post-class: 90.0
------
alison wong
pre-class: 75.0
post-class: 80.0
------
alona brequillo
pre-class: 85.0
post-class: 95.0
------
beverly sealey
pre-class: 40.0
post-class: 90.0
------
cherie pepper
pre-class: 80.0
post-class: 90.0
------
e sasso
pre-class: 90.0
post-class: 90.0
------
elsa wilson
pre-class: 60.0
post-class: 70.0
------
felicia pooler
pre-class: 80.0
post-class: nan
------
john niland
pre-class: 90.0
post-class: 90.0
------
julia hernandez-mejia
pre-class: 50.0
post-class: 75.0
------
mayra lindemann
pre-class: 60.0
post-class: 90.0
------
myrna gabriel
pre-class: 65.0
post-class: 95.0
------
orlando roman
pre-class: 65.0
post-class: 95.0
------
robert vargas
pre-class: 45.0
post-class: 95.0
------
roselynne mizner
pre-class: 80.0
post-class: 95.0
------
sitara shamim
pre-class: 85.0
post-class: 100.0
------


In [281]:
full_student_grades = pd.DataFrame([fullnames.fullname.to_list(), pre_class_scores, post_class_scores]).T
full_student_grades.columns = ['name', 'pre-class', 'post-class']
full_student_grades

,name,pre-class,post-class
0,alana chuong,55.0,90.0
1,alison wong,75.0,80.0
2,alona brequillo,85.0,95.0
3,beverly sealey,40.0,90.0
4,cherie pepper,80.0,90.0
5,e sasso,90.0,90.0
6,elsa wilson,60.0,70.0
7,felicia pooler,80.0,NaN
8,john niland,90.0,90.0
9,julia hernandez-mejia,50.0,75.0


### Save Results -> DataFrame -> Excel

In [282]:
out_path = os.path.join(cwd, 'output', program)
os.makedirs(out_path, exist_ok=True)
print('Folder created at: {}'.format(out_path))

Folder created at: /Users/kangchen/Documents/Projects/ors_auto_grading/output/powerpoint


#### Full student report

In [283]:
full_report_path = os.path.join(out_path, f'0_all_student_{program}_report.xlsx')

if os.path.exists(full_report_path):
    raise Exception(f'File already exists at: {full_report_path}\n'
                    'To create new report remove existing file')
else:    
    full_student_grades.to_excel(full_report_path, sheet_name=f'{program}_grades', index=False)
    if os.path.exists(full_report_path):
        print('Full student report created')

Full student report created


#### Answers

In [284]:
# check length of data
names.shape[0] == final_results.firstname.value_counts().count()

True

In [285]:
names.shape[0] == final_results.lastname.value_counts().count()

True

In [286]:
questions = pd.DataFrame(col_names_to_replace.copy(), index=col_names).T
questions.iloc[0,0] = 'Questions'
questions.iloc[:,1:5] = np.nan
questions

,timestamp,email,score,firstname,lastname,Q1,Q2,Q3,Q4,Q5,...,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
0,Questions,NaN,NaN,NaN,NaN,1. True or False: To apply a theme to all of y...,2. Which of the following areas can be used on...,3. Which of the following would you select whe...,4. Select all the ways you can format text in ...,5. Which graphical element(s) can you add from...,...,11. There are four view options that are acces...,12. What function can you use to group slides ...,13. True or False: Animations and Transitions ...,"14. What two groups allow you to control, cust...",15. True or False: Themes allow users to add p...,16. Which tab should you click on to display t...,17. What is meant by slide Transitions?,18. What is one of the primary functions of th...,19. True or False: Master View allows you to m...,"20. During a slide show, what happens if you p..."


In [287]:
answer_row.iloc[0,0] = 'Answers'
answer_row

,timestamp,email,score,firstname,lastname,Q1,Q2,Q3,Q4,Q5,...,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
0,Answers,NaN,NaN,NaN,NaN,False,Text Box,Slide Master view,All of the above.,All of the above,...,"Normal View, Slide Sorter View, Reading View, ...",Section,True,"Timing, Advanced Animation",True,Design Tab,They are animation-like effects that occur in ...,It is used for easily reorganizing slides in a...,True,Ends the slide show.


In [288]:
# concat all dataframes and save as excel file
n_files = 0

for i, row in names.iterrows():
  fname = row.FirstName
  lname = row.LastName

  cols = final_results.columns[final_results[final_results.firstname == fname].notna().any()]
  report = pd.concat([questions, answer_row, final_results[final_results.firstname == fname]], axis=0).reset_index(drop=True)[cols]
  report.fillna('-', inplace=True)
  report.drop(columns=['email', 'firstname', 'lastname'], inplace=True)
  report.rename(columns={'timestamp': 'Index'}, inplace=True)
  report.set_index('Index', inplace=True)

  file_path = os.path.join(out_path, f'{fname}_{lname}_{program}_report.xlsx')
  report.to_excel(file_path, f'{fname}_{lname}')
  
  if os.path.exists(file_path):
    print(f'{fname} {lname}: Report created')
    n_files += 1
  else:
    raise Exception(f'Failed to create report for: {fname} {lname}')
print('---------')
print(f'{n_files} reports created for: {program}')

alana chuong: Report created
alison wong: Report created
alona brequillo: Report created
beverly sealey: Report created
cherie pepper: Report created
e sasso: Report created
elsa wilson: Report created
felicia pooler: Report created
john niland: Report created
julia hernandez-mejia: Report created
mayra lindemann: Report created
myrna gabriel: Report created
orlando roman: Report created
robert vargas: Report created
roselynne mizner: Report created
sitara shamim: Report created
---------
16 reports created for: powerpoint


### Save Answer Key

In [289]:
answer_df = answer_row.iloc[:,5:]
questions_list = col_names_to_replace[5:]
answer_df.columns = questions_list

In [290]:
answer_path = os.path.join(out_path, f'0_{program}_answers.xlsx')
answer_df.to_excel(answer_path, f'{program}', index=False)

if os.path.exists(answer_path):
    print(f'{program}: Answer key saved')
else:
    raise Exception(f'Failed to save answer key for: {program}')

powerpoint: Answer key saved
